In [2]:
!unzip twitter.zip

Archive:  twitter.zip
   creating: twitter/
  inflating: twitter/twitter-test-neg.txt  
  inflating: twitter/twitter-test-pos.txt  
  inflating: twitter/twitter-train-neg.txt  
  inflating: twitter/twitter-train-pos.txt  


In [48]:
def openfile(filename):
    word_list = []
    f = open(filename)
    for line in f:
      word_list.append(line)
    return word_list

train_pos = []
train_neg = []
test_pos = []
test_neg = []
train_pos = openfile("twitter/twitter-train-pos.txt")
train_neg = openfile("twitter/twitter-train-neg.txt")
test_pos = openfile("twitter/twitter-test-pos.txt")
test_neg = openfile("twitter/twitter-test-neg.txt")



In [61]:
import pandas as pd
train_pos_df = pd.DataFrame(train_pos,columns = ["text"])
train_neg_df = pd.DataFrame(train_neg,columns = ["text"])
test_pos_df = pd.DataFrame(test_pos,columns = ["text"])
test_neg_df = pd.DataFrame(test_neg,columns = ["text"])

train_pos_df=train_pos_df.fillna("")
train_neg_df=train_neg_df.fillna("")
test_pos_df=test_pos_df.fillna("")
test_neg_df=test_neg_df.fillna("")


print(train_pos_df.head())

                                                text
0               Juuuuuuuuuuuuuuuuussssst Chillin!!\n
1         hmmmm.... i wonder how she my number @-)\n
2  Feeling strangely fine. Now I'm gonna go liste...
3                                "   goodbye exams\n
4  (: !!!!!! - so i wrote something last week. an...


In [62]:

ones = [1 for val in range(train_pos_df.shape[0])]
train_pos_df["label"] = ones 
ones = [1 for val in range(test_pos_df.shape[0])]
test_pos_df["label"] = ones 

zeros = [0 for val in range(train_neg_df.shape[0])]
train_neg_df["label"] = zeros
zeros = [0 for val in range(test_neg_df.shape[0])]
test_neg_df["label"] = zeros

print(train_pos_df.head())
print(train_neg_df.head())

                                                text  label
0               Juuuuuuuuuuuuuuuuussssst Chillin!!\n      1
1         hmmmm.... i wonder how she my number @-)\n      1
2  Feeling strangely fine. Now I'm gonna go liste...      1
3                                "   goodbye exams\n      1
4  (: !!!!!! - so i wrote something last week. an...      1
                                                text  label
0                 I missed the New Moon trailer...\n      0
1     i think mi bf is cheating on me!!!       T_T\n      0
2  Sunny Again        Work Tomorrow  :-|       TV...      0
3                   this weekend has sucked so far\n      0
4                             ok thats it you win.\n      0


In [63]:
train_df = pd.concat([train_pos_df, train_neg_df])
test_df = pd.concat([test_pos_df, test_neg_df])

print(train_df.shape)
print(test_df.columns)

(750007, 2)
Index(['text', 'label'], dtype='object')


In [53]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /home/nbuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/nbuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [66]:
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
def clean_text(text):
        text = text.lower()
        text = [word.strip(string.punctuation) for word in text.split()]
        text = [word for word in text if not any(c.isdigit() for c in word)]
        stop = stopwords.words("english")
        text = [word for word in text if word not in stop]
        text = [word for word in text if len(word) > 0]
        lemma = WordNetLemmatizer()
        text = [lemma.lemmatize(word) for word in text]
        text = [word for word in text if len(word) > 1]
        text = " ".join(text)
        return text

train_df["clean_text"] = train_df["text"].apply(lambda x:clean_text(x))
test_df["clean_text"] = test_df["text"].apply(lambda x:clean_text(x))

print(train_df.head())
print(test_df.head())

                                                text  label  \
0               Juuuuuuuuuuuuuuuuussssst Chillin!!\n      1   
1         hmmmm.... i wonder how she my number @-)\n      1   
2  Feeling strangely fine. Now I'm gonna go liste...      1   
3                                "   goodbye exams\n      1   
4  (: !!!!!! - so i wrote something last week. an...      1   

                                          clean_text  
0                   juuuuuuuuuuuuuuuuussssst chillin  
1                                hmmmm wonder number  
2  feeling strangely fine i'm gonna go listen sem...  
3                                       goodbye exam  
4  wrote something last week got call someone new...  
                                                text  label  \
0                          omg its already 7:30 :O\n      1   
1  handed in my uniform today . i miss you already\n      1   
2  thanks to all the haters up in my face all day...      1   
3  You're the only one who can see this

In [68]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tokenizer = CountVectorizer(analyzer='word')
tokenizer.fit(train_df["clean_text"])

x_train = tokenizer.transform(train_df["clean_text"])
x_valid = tokenizer.transform(test_df["clean_text"])

In [71]:
from sklearn.linear_model  import LogisticRegression 
from sklearn.metrics import f1_score

lr = LogisticRegression()
lr.fit(x_train, train_df["label"])

print(f1_score(test_df["label"], lr.predict(x_valid)))

0.7548678591322646
